In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community import embeddings
from langchain.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
import textwrap

In [2]:
loader = PyPDFDirectoryLoader("data")
the_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(the_text)

In [3]:
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="ollama_embeds",
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

/var/folders/dq/f61q7zlj4qb_kx8ly7hp73hh0000gp/T/ipykernel_50863/3918979500.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model='nomic-embed-text'),


In [4]:
llm = ChatGroq(
    api_key = "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",
    model="llama3-8b-8192"
)

In [5]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [6]:
response = rag_chain.invoke("Tell me about sugarcane agriculture")
print(textwrap.fill(response, width=80))

Based on the provided context, here are some key points about sugarcane
agriculture:  1. **Soil Requirements**: Sugarcane grows best on medium-heavy
soils, but can also be raised on lighter soils (sand) or heavy clay soils,
provided there is adequate irrigation available in the former type and drainage
is good in the latter type. 2. **Atmospheric Conditions**: Sugarcane grows well
at a temperature of 21° to 26.2°C, with an ideal temperature for germination
between 20-23.7°C. Temperatures above 32.2°C and below 18.3°C can arrest plant
growth. 3. **Sunlight**: Sugarcane loves warmth and sunshine, requiring long
days for vegetative growth and short days for reproductive growth. 4. **Water
Requirements**: Sugarcane needs optimum moisture during all stages of growth,
with water requirements varying from 200-300 mm depending on soil type and
weather conditions. Timely irrigations can result in more juice with high
sucrose contents. 5. **Irrigation**: The crop should be grown in areas of well